# Capstone Project: [testfit.io](https://blog.testfit.io/)
## Wrangle

In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import wrangle as wr

import warnings
warnings.filterwarnings("ignore")


Bad key "text.kerning_factor" on line 4 in
/usr/local/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
pd.set_option("display.max_columns", None)
plt.rc("figure", figsize=(16, 8))

## Stakeholder Notes
- Work with Commitments
- Activity Descriptions == New Construction, SubRehab, Additions, and Improvements
- Loan amount >= $1M:
    - Mortgage at time of Firm Commitment Issuance, Amendment or Reissuance?
    - Mortgage at Firm Commitment Issuance?
    - Final Mortgage Amount? **Probably this one...**
- Market == Large urban centers; meaning **cities** not just states; not rural one-off projects
- Types of FHA loan is irrelevant

In [ ]:
# commitments
# df  = pd.read_excel("Initi_Endores_Firm Comm_DB_FY06_FY20_Q2.xlsx", sheet_name="Firm Cmtmts, Iss'd and Reiss'd", header=6)
# print(df.shape)
# df.head()
df = pd.read_csv("hud_commitments.csv", index_col=0)
print(df.shape)
df.head()

In [ ]:
# df["Activity Description"].value_counts()

In [ ]:
# df.to_csv("hud_commitments.csv")

In [ ]:
# df.rename(columns={"Activity Group": "activity_group"}, inplace=True)

In [ ]:
# activity_groups filtered for groups of interest for clifton
# df = df[(df.activity_group == "NC/SR") | (df.activity_group == "Imp/Adds")]

In [ ]:
# only including loans greater than or equal to $1M
# df = df[df["Final Mortgage Amount"] >= 1000000]
# df.head()

In [ ]:
df.info()

In [ ]:
# df["Activity Description"].value_counts()

In [ ]:
# df["activity_group"].value_counts()

In [ ]:
df.rename(columns={"Facility Type": "facility_type"}, inplace=True)

In [ ]:
df.facility_type.value_counts()

In [ ]:
df.rename(columns={"FHA Number": "fha_number"}, inplace=True)

In [ ]:
df.groupby(["Project State"]).fha_number.count().sort_values().plot.barh()
plt.show()

In [ ]:
# df[df["Final Mortgage Amount"] >= 1000000].shape

In [ ]:
df.rename(columns={"Final Mortgage Amount": "final_mortgage"}, inplace=True)

In [ ]:
df.groupby(["Project State"]).final_mortgage.sum().sort_values().plot.barh()

In [ ]:
df["Project City"].value_counts().head(20)

In [ ]:
df.groupby(["Project City", "Fiscal Year of Firm Commitment"]).size().sort_values()

In [ ]:
df.groupby(["Project City", "Fiscal Year of Firm Commitment"])["final_mortgage"].sum().sort_values()

In [ ]:
df[df["Project City"] == "Austin"].groupby("Fiscal Year of Firm Commitment")["final_mortgage"].sum()

In [ ]:
df[df["Project City"] == "Minneapolis"].groupby("Fiscal Year of Firm Commitment")["final_mortgage"].sum()

In [ ]:
# csv works and can be read
df = pd.read_csv("hud_commitments.csv", index_col=0)
df.head()

In [ ]:
df[df["Project City"] == "Austin"].groupby("Fiscal Year of Firm Commitment")["Final Mortgage Amount"].sum()

In [ ]:
df[df["Project City"] == "Austin"].groupby("Fiscal Year of Firm Commitment")["Final Mortgage Amount"].count()

In [ ]:
df[df["Project City"] == "Austin"].groupby("Fiscal Year of Firm Commitment")["Final Mortgage Amount"].size()

In [ ]:
df.shape

In [ ]:
# let's see the number of unique values for earch column
# for col in df.columns:
#     unique = len(df[col].unique())
#     if unique > 10:
#         print()
#         print(f'{col} has {unique} values')
#     else:
#         print()
#         print(f'{col} value count:')
#         print(df[col].value_counts())

There are hospitals in this data.

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
# df["Date of Firm Issue"] = pd.to_datetime(df["Date of Firm Issue"])
# df["Date of Firm Issue"].dt.year

---
## Wrangle

In [ ]:
df = wr.wrangle_hud()
print(df.shape)
df.head()

In [ ]:
df.info()

---
## Explore
### What are the characteristics of loan practices in Houston (2009), Seattle (2010), and Dallas (2012)?
- Do these markets follow a similar trajectory in the years leading up the boom?

In [ ]:
# fmt-clipboard function from terminal == black formatting
# df[df.project_city == "Houston"].groupby(["date_of_firm_commitment_activity"])[
#     "final_mortgage_amount"
# ].sum().reset_index().sort_values(by="date_of_firm_commitment_activity")

In [ ]:
# clifton's time machine markets
threshold_cities = ["Houston", "Seattle", "Dallas"]

# loop through 
for city in threshold_cities:
    # subset for city
    date_index_df = df[df.project_city == city].sort_values(
        "date_of_firm_commitment_activity"
    ).set_index("date_of_firm_commitment_activity")

    # sum of the final_mortgage_amount by date_of_firm_commitment_activity
    date_index_df = (
        date_index_df.groupby(["date_of_firm_commitment_activity", "activity_description"])[
            "final_mortgage_amount"
        ]
        .sum()
        .reset_index()
    )
    
    date_index_df.set_index("date_of_firm_commitment_activity", inplace=True)
    
    # plot
    sns.lineplot(data=date_index_df, x=date_index_df.index, y="final_mortgage_amount", hue="activity_description")
    plt.title(f"Final Mortgage Amount Over Time: {city}")
    plt.xlabel("Date of Firm Commitment Activity")
    plt.ylabel("Final Mortgage Amount")
    plt.show()
    
    yearly_avg = date_index_df.resample('Y').mean()
    sns.lineplot(data=yearly_avg, x=yearly_avg.index, y="final_mortgage_amount")
    plt.title(f"Yearly Average Final Mortgage Amount: {city}")
    plt.xlabel("Date of Firm Commitment Activity")
    plt.ylabel("Final Mortgage Amount")
    plt.show()
    
    two_year_avg = date_index_df.resample('2Y').mean()
    sns.lineplot(data=two_year_avg, x=two_year_avg.index, y="final_mortgage_amount")
    plt.title(f"Two-Year Average Final Mortgage Amount: {city}")
    plt.xlabel("Date of Firm Commitment Activity")
    plt.ylabel("Final Mortgage Amount")
    plt.show()

**Takeaway**: What I observe in the charts above is that for the years that Clifton would have liked to enter into a market, the total final mortgage production for that city droped to $10M or less. We need to know from Clifton which markets he know's it's too late to target to derive a more general metric for labeling markets.

### Hypothesis Testing

$H_0$: The mean final mortgage amount is the same across all activity descriptions

$H_a$: The mean final mortgage amount is different depending on the activity description

In [ ]:
df.activity_description.value_counts()

In [ ]:
# 99% confidence level
alpha = .01

refi = df[df.activity_description == "Refinance"].final_mortgage_amount
nc = df[df.activity_description == "New Construction"].final_mortgage_amount
sr = df[df.activity_description == "SubRehab"].final_mortgage_amount
purch = df[df.activity_description == "Purchase"].final_mortgage_amount
add = df[df.activity_description == "Additions"].final_mortgage_amount
imp = df[df.activity_description == "Improvements"].final_mortgage_amount
ol = df[df.activity_description == "Operating Loss"].final_mortgage_amount

stat, pvalue = stats.f_oneway(refi, nc, sr, purch, add, imp, ol)

print(f"statistic = {stat}")
print(f"  p-value = {pvalue}")
print()
if pvalue < alpha:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")

In [ ]:
plt.figure(figsize=(16, 8))

sns.boxplot(data=df, x="activity_description", y="final_mortgage_amount")
plt.xlabel("Activity Description")
plt.ylabel("Final Mortgage Amount")
plt.show()

**Takeaways**: Looks like there are a lot of outliers that are skewing the data. However, we can clearly see that the new construction mortgages have the highest mean among the population.

$H_0$: The mean final mortgage amount is the same across all facility types

$H_a$: The mean final mortgage is different depending on the facility type

---
#### Detour To Standardize `project_city` Names

In [ ]:
pd.DataFrame(df.groupby(["project_city", "fiscal_year_of_firm_commitment"])["final_mortgage_amount"].sum().sort_values(ascending=False).head(50))

In [ ]:
df.project_city.unique()

In [ ]:
# def standardize_city_name(date):
#     return re.sub(r"(\d{2})/(\d{2})/(\d{2})", r"20\3-\1-\2", date)

In [ ]:
# regex = br""

# df["project_city"] = df.project_city.str.replace()

In [ ]:
# (^[a-z]+)(.)([a-z]+)

In [ ]:
# (^[A-Z][a-z]+)(\s?)([A-Z][a-z]+)

In [ ]:
# df.groupby(["project_city", "fiscal_year_of_firm_commitment"])["final_mortgage_amount"].sum()

In [ ]:
# number of unique cities before project_city string transformations
# len(df.project_city.unique())

In [ ]:
# df["project_city"] = df.project_city.str.strip()

In [ ]:
# df.groupby(["project_city", "fiscal_year_of_firm_commitment"])["final_mortgage_amount"].sum()

In [ ]:
# number of unique cities after project_city .strip()
# len(df.project_city.unique())

In [ ]:
# df["project_city"] = df.project_city.str.title()

In [ ]:
df.groupby(["project_city", "fiscal_year_of_firm_commitment"])["final_mortgage_amount"].sum()

In [ ]:
# number of unique cities after project_city .title() case
len(df.project_city.unique())

In [ ]:
df.groupby(["project_city", "fiscal_year_of_firm_commitment"])[
    "final_mortgage_amount"
].sum().sort_values(ascending=False).head(50)

---
### Back on Track with Hypothesis Testing
$H_0$ : The mean final mortgage amount is the same across all facility types

$H_a$ : The mean final mortgage is different depending on the facility type

In [ ]:
df.facility_type.value_counts()

In [ ]:
# 99% confidence level
alpha = .01

apts_coops = df[df.facility_type == "Apts/Coops"].final_mortgage_amount
apts = df[df.facility_type == "Apts"].final_mortgage_amount 
nursing_icf = df[df.facility_type == "Nursing/ICF"].final_mortgage_amount # intermediate care facility
alf = df[df.facility_type == "Asst'd Livg"].final_mortgage_amount # assisted living facility 
coop_apts_mh_alf = df[df.facility_type == "Coop/Apts/Mobile Home Pk/ALF"].final_mortgage_amount
coop = df[df.facility_type == "Coop"].final_mortgage_amount
bc = df[df.facility_type == "Board & Care"].final_mortgage_amount
hosp = df[df.facility_type == "Hospital"].final_mortgage_amount
health_care = df[df.facility_type == "232 Health Care-NH/ICF, ALF or B&C"].final_mortgage_amount
mobile_home = df[df.facility_type == "Mobile Home Pk"].final_mortgage_amount
sro = df[df.facility_type == "SRO"].final_mortgage_amount # single room occupancy 

stat, pvalue = stats.f_oneway(
    apts_coops,
    apts,
    nursing_icf,
    alf,
    coop_apts_mh_alf,
    coop,
    bc,
    hosp,
    health_care,
    mobile_home,
    sro
)


print(f"statistic = {stat}")
print(f"  p-value = {pvalue}")
print()
if pvalue < alpha:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")

In [ ]:
plt.figure(figsize=(16, 8))

sns.boxplot(data=df, x="facility_type", y="final_mortgage_amount")
plt.xlabel("Activity Description")
plt.ylabel("Final Mortgage Amount")
plt.show()

**Takeaway**: I think we should group some of these facilities together. There seems to be a lot of overlap. Based on the visualization above, Hospitals seem to have the highest variance and mean mortgage amount.